In [807]:
import warnings
warnings.filterwarnings('ignore')

In [808]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# Pipelines and grid search 

Grid search is a technique used in machine learning to find the best hyperparameters for a given model. Hyperparameters are parameters that are not learned from the data but are set before training the model. These can include things like learning rate, regularization strength, number of trees in a random forest, kernel type in a support vector machine, etc.
How Grid Search Works:

    Specify the hyperparameters to tune: You first decide which hyperparameters you want to optimize and define a list of possible values for each.

    Create a grid: The grid consists of all possible combinations of hyperparameters. For example, if you want to tune two hyperparameters, learning rate and the number of trees in a forest, the grid might look like this:
        Learning rate: [0.001, 0.01, 0.1]
        Number of trees: [10, 50, 100]

    Evaluate all combinations: Grid search systematically evaluates all combinations of hyperparameters by training the model with each combination, often using cross-validation to assess the model’s performance.

    Select the best combination: Once all combinations are tested, the hyperparameter combination that provides the best performance (e.g., highest accuracy, lowest error) is selected.

```python

# Grid Search Random example

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the model
model = RandomForestClassifier()

# Specify the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 500, 1000]
    'max_depth': [10, 20, None]
}

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters
print(grid_search.best_params_)


```

Pros:

    Simple to implement and easy to understand.
    Guarantees finding the best hyperparameter combination within the defined grid.

Cons:

    Can be computationally expensive, especially if the search space is large, because it evaluates all combinations.
    Doesn’t scale well to models with many hyperparameters.

# Pipelines 

In machine learning, a pipeline is a way of organizing and streamlining the various steps in a machine learning workflow. It ensures that all steps, from data preprocessing to model evaluation, are executed in a consistent and reproducible manner.
Components of a Pipeline:

    Data Preprocessing: This includes steps like data cleaning (handling missing values), scaling or normalizing features, encoding categorical variables, and feature selection.

    Model Training: The actual machine learning algorithm (e.g., decision trees, support vector machines) is applied to the processed data to build a model.

    Model Evaluation: This step involves evaluating the model’s performance using metrics like accuracy, precision, recall, etc., typically with a validation set or using cross-validation.

Why Use Pipelines?

    Streamlined Workflow: Pipelines allow you to chain multiple steps (like preprocessing and model training) together into a single object. This reduces the risk of errors when manually performing each step individually.

    Consistency: With a pipeline, you ensure that the same preprocessing steps are applied to both the training and testing data, which is crucial for model generalization.

    Reusability: Pipelines can be reused and shared with others, making it easier to apply the same sequence of operations to different datasets.

    Ease of Hyperparameter Tuning: When performing grid search or other hyperparameter optimization methods, pipelines ensure that all transformations are applied to each fold of the data in the correct order.

```python

# pipeline example 

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Sample data (features and target)
X = ...  # feature matrix
y = ...  # target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ("ohe",Onehotencoder())
    ('std', StandardScaler()),  # Step 1: Standardize the features
    ('clf', RandomForestClassifier())  # Step 2: Train a Random Forest model
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = (y_pred == y_test).mean()
print(f'Accuracy: {accuracy}')


```

# Pipelines with grid search

```python 

from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'clf__n_estimators': [50, 100],  # 'classifier' is the RandomForest model
    'clf__max_depth': [10, 20]
}

# Perform grid search on the pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get best parameters
print("Best parameters:", grid_search.best_params_)

```

## Logistic regression without piplines

In [809]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
# data loading
df =  pd.read_csv("./data/titanic_data.csv",index_col="PassengerId")
df.head()

In [ ]:
df.info()

Age and Cabin contain null values  we drop cabin and impute Age 

In [812]:
df.drop("Cabin",axis=1,inplace=True)

We also drop Embarked and Ticket

In [813]:
df.drop(["Embarked","Ticket","Name"],axis=1,inplace=True)

In [814]:
orginal_df = df

In [ ]:
imputer =  SimpleImputer(strategy="mean")
df["Age"] = imputer.fit_transform(df[["Age"]])
df.info()

In [ ]:
df.head()

### OHE Categorical columns 

In [ ]:
df =  pd.get_dummies(df,columns=["Sex"],drop_first=True,dtype=int)
df.head()

In [ ]:
df.info()

In [ ]:
X = df.drop("Survived",axis =1)
y = df["Survived"]
X.head()

In [ ]:
X.info()

In [ ]:
#stardaize numerical colums 
scaler = MinMaxScaler()
X[["Age","Parch","Fare"]] = scaler.fit_transform(X[["Age","Parch","Fare"]])
X.head()

In [822]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=20,random_state=42)

In [823]:
model =  LogisticRegression(max_iter=1000)

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_pred=y_pred,y_true=y_test))

## With a grid Search

In [825]:
param_grid = {
    'C': [0.1, 1, 10],            # Regularization strength (Inverse of regularization strength)
    'solver': ['liblinear', 'saga'],  # Optimization algorithms
    'penalty': ['l2', 'l1'],         # Regularization types
    'class_weight': [None, 'balanced']  # Handle imbalanced classes (optional)
}# Define the hyperparameters to tune


In [826]:
# Set up GridSearchCV

grid_search = GridSearchCV(estimator=LogisticRegression(),param_grid=param_grid,cv=20)


In [ ]:
# Fit the model with the best hyperparameters
grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.predict(X_test)

In [ ]:
# Output the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score: {:.2f}".format(test_score))

## With Pipelines 

In [831]:
# import pipelines
from sklearn.pipeline import Pipeline


In [ ]:
# recheck data 
df.head()


In [833]:
# re assign X and y
X=df.drop("Survived",axis=1)
y=df["Survived"]

In [834]:
# train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=20,random_state=42)

In [835]:
# Make Pipeline for stardadization and modeling

pipe = Pipeline([
    ("one",MinMaxScaler()),
    ("model",LogisticRegression())
])



In [836]:
# fit pipeline and make predictions
pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)


In [ ]:
print(classification_report(y_pred=y_pred,y_true=y_test))

## Pipelines with Grid Search

In [838]:
# redifine Param grid
param_grid = {
    'model__C': [0.1, 1, 10],            # Regularization strength (Inverse of regularization strength)
    'model__solver': ['liblinear', 'saga'],  # Optimization algorithms
    'model__penalty': ['l2', 'l1'],         # Regularization types
    'model__class_weight': [None, 'balanced']  # Handle imbalanced classes (optional)
}

In [839]:
# define a grid

grid = GridSearchCV(estimator=pipe,param_grid=param_grid)

In [840]:
# fit and get best param
grid.fit(X_train,y_train)

y_pred = grid.predict(X_test)

In [ ]:
print(classification_report(y_pred=y_pred,y_true=y_test))

## changing models /Steps in a pipeline


In [ ]:
# switching to Desciscion tree
from sklearn.tree import DecisionTreeClassifier
pipe.set_params(one=StandardScaler())
pipe.set_params(model=DecisionTreeClassifier())



In [843]:
# fit and make predictions 
pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)



In [ ]:
print(classification_report(y_true=y_test,y_pred=y_pred))

In [845]:
# Todo use swith to any other model and also do a grid search 

## Column Tranformers 

A **ColumnTransformer** in machine learning is used to apply different preprocessing techniques to different subsets of columns (features) in a dataset. It allows you to transform numerical and categorical columns with different operations, such as scaling numerical data or encoding categorical data, in a clean and efficient way.

### Benefits:
1. **Streamlined preprocessing**: You can apply different transformations to different columns in a single, unified step.
2. **Cleaner code**: Organizes preprocessing tasks and avoids manually separating data by column types.
3. **Flexibility**: You can specify custom transformations for each set of columns (e.g., scaling for numerical columns, one-hot encoding for categorical columns).
4. **Improved pipeline integration**: It integrates well within machine learning pipelines, ensuring consistency when training and testing models.

In [846]:
# imports 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer

Original data inspection

In [ ]:
orginal_df.head()

Custom Function

In [855]:
# custom function 
def plus_one(x):
    return x+50

## Tranformers

In [849]:
# Creating tranformer






In [850]:
# Pipeline


In [851]:
# re assign X and y
X=orginal_df.drop("Survived",axis=1)
y=orginal_df["Survived"]

In [852]:
# train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=20,random_state=42)

In [853]:
# fit and make predictions 


In [ ]:
print(classification_report(y_true=y_test,y_pred=y_pred))